In [1]:
import random

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt
import math
from d2l import torch as d2l

import operator
from functools import reduce
from functools import partial
from timeit import default_timer
from utilities3 import *

from Adam import Adam

torch.manual_seed(0)
np.random.seed(0)

In [2]:
################################################################
#  1d fourier layer
################################################################
class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes1):
        super(SpectralConv1d, self).__init__()

        """
        1D Fourier layer. It does FFT, linear transform, and Inverse FFT.    
        """

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes1 = modes1  # Number of Fourier modes to multiply, at most floor(N/2) + 1

        self.scale = (1 / (in_channels * out_channels))
        self.weights1 = nn.Parameter(
            self.scale * torch.rand(in_channels, out_channels, self.modes1, dtype=torch.cfloat))

    # Complex multiplication
    def compl_mul1d(self, input, weights):
        # (batch, in_channel, x ), (in_channel, out_channel, x) -> (batch, out_channel, x)
        return torch.einsum("bix,iox->box", input, weights)

    def forward(self, x):
        batchsize = x.shape[0]
        # Compute Fourier coeffcients up to factor of e^(- something constant)
        x_ft = torch.fft.rfft(x)
        # Multiply relevant Fourier modes
        out_ft = torch.zeros(batchsize, self.out_channels, x.size(-1) // 2 + 1, device=x.device, dtype=torch.cfloat)
        out_ft[:, :, :self.modes1] = self.compl_mul1d(x_ft[:, :, :self.modes1], self.weights1)
        # Return to physical space
        x = torch.fft.irfft(out_ft, n=x.size(-1))
        return x

class FNO1d_block(nn.Module):
    '''FNO块，其中未实现激活函数'''
    def __init__(self, modes, width):
        super().__init__()
        self.modes1 = modes
        self.width = width
        self.conv0 = SpectralConv1d(self.width, self.width, self.modes1)
        self.w0 = nn.Conv1d(self.width, self.width, 1)

    def forward(self, x):
        x1 = self.conv0(x)
        x2 = self.w0(x)
        x = x1 + x2
        return x

class Permute(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.dim = args

    def forward(self, x):
        x = x.permute(dims=self.dim)
        return x

class Get_grid(nn.Module):
    def __init(self):
        super().__init__()

    def get_grid(self, shape, device):
        batchsize, size_x = shape[0], shape[1]
        gridx = torch.tensor(np.linspace(0, 2 * math.pi, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1).repeat([batchsize, 1, 1])
        return gridx.to(device)

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        return x

# class convd_0(nn.Module):
#     def __init__(self, out_channels, num_padding, num_kernel):
#         super().__init__()
#         self.out_channels = out_channels
#         self.num_padding = num_padding
#         self.num_kernel = num_kernel
        
#     def forward(self, x):
#         if x.shape[1] == 2:
#             x = F.pad(x, self.num_padding).unsqueeze(dim=1)
#         else:
#             x = F.pad(x, self.num_padding).unsqueeze(dim=2)
# #         elif len(x.shape) == 4:
# #             x = F.pad(x, num_padding)
#         convd_layer = nn.Conv2d(in_channels=x.shape[1], out_channels=self.out_channels, 
#                                 kernel_size=self.num_kernel, device=x.device)
#         convd_out = convd_layer(x)
#         if convd_out.shape[2] == 1:
#             convd_out = convd_out.squeeze(dim=2)
#         return convd_out.to(x.device)

#     def convd_1(self, x, out_channel, num_padding, num_kernel):
#         if len(x.shape) == 3:
#             x = F.pad(x, num_padding).unsqueeze(dim=2)
#         elif len(x.shape) == 4:
#             x = F.pad(x, num_padding)
#         in_channel=x.shape[1]
#         convd_layer = nn.Conv2d(in_channel, out_channel, kernel_size=num_kernel, device=x.device)
#         convd_out = convd_layer(x)
#         if convd_out.shape[2] == 1:
#             convd_out = convd_out.squeeze(dim=2)
#         return convd_out.to(device)



In [3]:
################################################################
#  configurations
################################################################
ntrain = 10000
ntest = 1025

sub = 2 ** 3  # subsampling rate
h = 2 ** 13 // sub  # total grid size divided by the subsampling rate
s = h

batch_size = 10
learning_rate = 0.001

epochs = 10
step_size = 50
gamma = 0.5

modes = 16
width = 64

################################################################
# read data
################################################################

# Data is of the shape (number of samples, grid size)
dataloader = MatReader('../data/possion_data3.2_10000_1024.mat')
x_train = dataloader.read_field('f_train')[:ntrain, :]
y_train = dataloader.read_field('u_train')[:ntrain, :]
x_test = dataloader.read_field('f_test')[:ntest, :]
y_test = dataloader.read_field('u_test')[:ntest, :]

x_train = x_train.reshape(ntrain, -1, 1)
x_test = x_test.reshape(ntest, -1, 1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size,
                                          shuffle=False)

model=nn.Sequential(Get_grid(), nn.Linear(2, width), Permute(0,2,1),
                    FNO1d_block(modes, width), nn.ReLU(),
                    FNO1d_block(modes, width), nn.ReLU(),
                    FNO1d_block(modes, width), nn.ReLU(),
                    Permute(0,2,1), nn.Linear(width, 128), nn.ReLU(), nn.Linear(128, 1))
model = model.cuda()
model.apply(xavier)
print('number of params:', count_params(model))


number of params: 414337


In [4]:
################################################################
# training and evaluation
################################################################
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

myloss = LpLoss(size_average=False)
for ep in range(epochs):
    model.train()
#     animator = Animator(xlabel='epoch', xlim=[1, epochs],legend=['train_l2'])
    t1 = default_timer()
    # train_mse = 0
    # train_l2 = 0
    metric_train = Accumulator(3) #累加每epoch，训练集上的mseloss,训练集上的l2_loss,测试集上的l2_loss
    for x, y in train_loader:
        x, y = x.to(device='cuda'), y.to(device='cuda')

        optimizer.zero_grad()
        out = model(x)

        mse = F.mse_loss(out.view(batch_size, -1), y.view(batch_size, -1), reduction='mean')
        l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1))
        l2.backward() # use the l2 relative loss
        optimizer.step()

        metric_train.add(mse.item(), l2.item(), 0)
        # train_mse += mse.item()
        # train_l2 += l2.item()

    scheduler.step()
    model.eval()
    # test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.cuda(), y.cuda()

            out = model(x)
            test_l2 = myloss(out.view(batch_size, -1), y.view(batch_size, -1)).item()
            metric_train.add(0, 0, test_l2)

    train_mse = metric_train[0] / len(train_loader)
    train_l2 = metric_train[1] / ntrain
    test_l2 = metric_train[2] / ntest

    t2 = default_timer()
    print(ep, t2-t1, train_mse, train_l2, test_l2)
#     animator.add(ep + 1, train_l2)

# torch.save(model.state_dict(), 'model/params_possion_data2.5_10000_1024_fix')

# pred = torch.zeros(y_test.shape)
# index = 0
# test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=1, shuffle=False)
# with torch.no_grad():
#     for x, y in test_loader:
#         test_l2 = 0
#         x, y = x.cuda(), y.cuda()
#
#         out = model(x).view(-1)
#         pred[index] = out
#
#         test_l2 += myloss(out.view(1, -1), y.view(1, -1)).item()
#         if index % 10 == 0:
#             print(index, test_l2)
#         index = index + 1

# scipy.io.savemat('pred/burger_test.mat', mdict={'pred': pred.cpu().numpy()})



0 13.7066567 17146.20271066284 0.05830602565854788 0.03521356687313173


KeyboardInterrupt: 

In [ ]:
num=random.randint(1,ntrain)
print(num)
x_random=x_train[num,:,:].reshape(1,-1,1)
y_random=y_train[num,:]
y_pre=model(x_random.cuda())
plt.plot(np.linspace(0,2*math.pi,len(y_random)), y_random, color="blue", linewidth=3.0, linestyle="-", label="-")
plt.plot(np.linspace(0,2*math.pi,len(y_random)), y_pre.view(-1).to(torch.device("cpu")).detach().numpy(), color="red", linewidth=1.0, linestyle="solid", label="--")
plt.show()